In [2]:
import json

file_path = '../stage1_problems/STAGE1_1.json'

with open(file_path, 'r') as file:
    data = json.load(file)



In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
class Order:
    def __init__(self, order_info):
        # [ORD_ID, ORD_TIME, SHOP_LAT, SHOP_LON, DLV_LAT, DLV_LON, VOL, COOK_TIME, DLV_DEADLINE]
        self.id = order_info[0]
        self.order_time = order_info[1]
        self.shop_lat = order_info[2]
        self.shop_lon = order_info[3]
        self.dlv_lat = order_info[4]
        self.dlv_lon = order_info[5]
        self.cook_time = order_info[6]
        self.volume = order_info[7]
        self.deadline = order_info[8]

        self.ready_time = self.order_time + self.cook_time

    def __repr__(self) -> str:
        return f'Order([{self.id}, {self.order_time}, {self.shop_lat}, {self.shop_lon}, {self.dlv_lat}, {self.dlv_lon}, {self.volume}, {self.cook_time}, {self.deadline}])'

# 배달원 class
class Rider:
    def __init__(self, rider_info):
        # [type, speed, capa, var_cost, fixed_cost, service_time, available number]
        self.type = rider_info[0]
        self.speed = rider_info[1]
        self.capa = rider_info[2]
        self.var_cost = rider_info[3]
        self.fixed_cost = rider_info[4]
        self.service_time = rider_info[5]
        self.available_number = rider_info[6]
    
    def __repr__(self) -> str:
        return f'Rider([{self.type}, {self.speed}, {self.capa}, {self.var_cost}, {self.fixed_cost}, {self.service_time}, {self.available_number}])'
    
    # 주어진 거리에 대한 배달원 비용 계산
    # = 배달원별 고정비 + 이동거리로 계산된 변동비
    def calculate_cost(self, dist):
        return self.fixed_cost + dist / 100.0 * self.var_cost

# 묶음 주문 정보
class Bundle:
    def __init__(self, all_orders, rider, shop_seq, dlv_seq, total_volume, total_dist, feasible=True):
        self.rider = rider
        self.all_orders = all_orders
        self.feasible = feasible
        self.shop_seq = shop_seq
        self.dlv_seq = dlv_seq
        self.total_volume = total_volume
        self.total_dist = total_dist
        self.update_cost()

    # 묶음 주문의 비용 update
    def update_cost(self):
        self.cost = self.rider.calculate_cost(self.total_dist)
        self.cost_per_ord = self.cost / len(self.shop_seq)


    def __repr__(self) -> str:
        return f'Bundle(all_orders, {self.rider.type}, {self.shop_seq}, {self.dlv_seq}, {self.total_volume}, {self.feasible})'


In [18]:
riders_data = data['RIDERS']
orders_data = data['ORDERS']
dist_data = data['DIST']

riders_columns = ['Transport', 'Speed', 'Capacity', 'RestTime', 'Cost', 'Income', 'Satisfaction']
riders_df = pd.DataFrame(riders_data, columns=riders_columns)

orders_columns = ['OrderID', 'Time', 'StartLat', 'StartLong', 'EndLat', 'EndLong', 'Distance', 'TimeWindow', 'deadline']
orders_df = pd.DataFrame(orders_data, columns=orders_columns)

dist_df = pd.DataFrame(dist_data)

In [26]:
riders_df

,Transport,Speed,Capacity,RestTime,Cost,Income,Satisfaction
0,BIKE,5.291005,100,60,8000,120,20
1,WALK,1.322751,70,30,8000,120,30
2,CAR,4.232804,200,100,6000,180,100


In [19]:
K = data['K']

all_orders = [Order(order_info) for order_info in data['ORDERS']]
all_riders = [Rider(rider_info) for rider_info in data['RIDERS']]

dist = np.array(data['DIST'])
for r in all_riders:
    r.T = np.round(dist/r.speed + r.service_time)

In [28]:
len(all_orders)

100

In [29]:
all_riders

[Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 20]),
 Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30]),
 Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 100])]